### 🔶BAGOFWORDS con pyspark
* Alumna : Leydi Diana Choque Sarmiento

In [5]:
import math

In [1]:
def BOW(Corpus):
  
  x = Corpus.strip().lower().split()
  return list(filter(lambda x: len(x)>2,x)) # solo palabras mayor a dos de tamaño

In [17]:
CORPUS = ["balon balon balon futbol futbol liga liga liga ronaldo ronaldo ronaldo messi",
            "futbol futbol futbol futbol futbol ronaldo ronaldo ronaldo ronaldo messi messi",
            "balon balon futbol futbol futbol futbol futbol futbol messi messi messi messi",
            "politica politica politica pp pp pp pp pp pp rajoy rajoy rajoy rajoy rajoy",
            "politica politica politica politica pp pp psoe psoe psoe psoe zapatero zapatero rajoy",
            "politica politica politica psoe psoe psoe psoe zapatero zapatero zapatero zapatero ",
            "dinero fmi fmi fmi fmi fmi ue ue ue ue pib pib pib ibex ibex",
            "zapatero rajoy dinero dinero dinero dinero fmi fmi fmi fmi ue ue ue ue pib",
            "pp psoe zapatero rajoy dinero dinero dinero dinero fmi fmi fmi fmi ue ue ue ",
            "futbol politica pib",
            "futbol liga politica zapatero rajoy"]

Corpus_words = sc.parallelize(CORPUS,6)
Corpus_words = Corpus_words.map(BOW)
Corpus_words.take(3)

[['balon',
  'balon',
  'balon',
  'futbol',
  'futbol',
  'liga',
  'liga',
  'liga',
  'ronaldo',
  'ronaldo',
  'ronaldo',
  'messi'],
 ['futbol',
  'futbol',
  'futbol',
  'futbol',
  'futbol',
  'ronaldo',
  'ronaldo',
  'ronaldo',
  'ronaldo',
  'messi',
  'messi'],
 ['balon',
  'balon',
  'futbol',
  'futbol',
  'futbol',
  'futbol',
  'futbol',
  'futbol',
  'messi',
  'messi',
  'messi',
  'messi']]

### TF

In [4]:
def TF_Implementado(Corpus):   
    # Calculamos words del corpus
    cant_words=len(Corpus)
    # Obtener lista sin las repeticiones del corpus
    cant_sin_duplicar=list(set(Corpus))

    #Obtenemos el tf para cada palabra
    mapeo=list(map(lambda x:(x,Corpus.count(x)),cant_sin_duplicar))
    
    tf=list(map(lambda x:(x[0],1+math.log10(x[1]/cant_words)),mapeo))# Calculamos logaritmicamente

    return (tf)

In [7]:
Corpus_words = sc.parallelize(CORPUS,3)
Corpus_words = Corpus_words.map(BOW)
Corpus_TF=Corpus_words.map(TF_Implementado)
Corpus_TF.take(2)

[[('ronaldo', 0.3979400086720376),
  ('futbol', 0.22184874961635637),
  ('messi', -0.07918124604762489),
  ('liga', 0.3979400086720376),
  ('balon', 0.3979400086720376)],
 [('ronaldo', 0.5606673061697374),
  ('futbol', 0.6575773191777937),
  ('messi', 0.2596373105057561)]]

### IDF

In [8]:
def IDF_Implementacion(Document,corpus):
    Documento_New=Document.flatMap(lambda x:x)
    Documento_New=Documento_New.map(lambda x:x[0]).distinct()
     
    total=corpus.count()
    corpus_collection=corpus.collect()
     
    IDF=Documento_New.map(lambda x:list(map(lambda y:(x,1 if x in y else 0),corpus_collection)))
    IDF_new=IDF.flatMap(lambda x:x)
    IDF_new=IDF_new.groupByKey().map(lambda x:(x[0],sum(x[1])))
    IDF_new=IDF_new.map(lambda x:(x[0],math.log10(1+total/x[1]))) 
    return IDF_new

### IDF de cada palabra

In [10]:
def IDF_word(word,corpusIDF):
         
        L=corpusIDF
        EL=list(filter(lambda x:x[0]==word,L))
        return EL[0][1]

### TF_IDF

In [14]:
def TF_IDF(TextoTF,TextoIDF):
    """Funcion que calcula el IDF para cada Doc de TF"""
    corpusIDF=TextoIDF.collect()
    TFIDF=TextoTF.map(lambda x:list(map((lambda y:(y[0],y[1]*IDF_word(y[0],corpusIDF))),x)))
    return  TFIDF

In [18]:
Corpus_words = sc.parallelize(CORPUS,3)
Corpus_words = Corpus_words.map(BOW)
Corpus_TF = Corpus_words.map(TF_Implementado)
Corpus_IDF = IDF_Implementacion(Corpus_TF,Corpus_words)
Corpus_TF_IDF = TF_IDF(Corpus_TF,Corpus_IDF)
Corpus_TF_IDF.collect()

[[('ronaldo', 0.32349074819207313),
  ('futbol', 0.11206689105900068),
  ('messi', -0.052972790530610464),
  ('liga', 0.32349074819207313),
  ('balon', 0.32349074819207313)],
 [('ronaldo', 0.4557739418183488),
  ('futbol', 0.33217516852632445),
  ('messi', 0.17369912131819806)],
 [('futbol', 0.3530846825366045),
  ('messi', 0.3498094262116576),
  ('balon', 0.18034381171765268)],
 [('rajoy', 0.40203877350660605), ('politica', 0.28997188244760536)],
 [('zapatero', 0.13115578177302142),
  ('rajoy', -0.020909514010280014),
  ('psoe', 0.3750902296893321),
  ('politica', 0.283221077556323)],
 [('zapatero', 0.283221077556323),
  ('psoe', 0.3750902296893321),
  ('politica', 0.22010827746732375)],
 [('ibex', 0.280195716272056),
  ('fmi', 0.43992368553450556),
  ('pib', 0.291505367693982),
  ('dinero', -0.02769198705293586)],
 [('fmi', 0.3750902296893321),
  ('rajoy', -0.020909514010280014),
  ('dinero', 0.3750902296893321),
  ('zapatero', -0.020909514010280014),
  ('pib', -0.02769198705293586)]